In [43]:
from datetime import date, datetime, timedelta
import requests
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json
import re

In [7]:
train = pd.read_csv('train_pred.csv', sep='\t')
test = pd.read_csv('test_pred.csv', sep='\t')

combined = pd.concat([train, test])

combined['departure'] = pd.to_datetime(combined['Departure time'])
combined['departure']

0     2012-01-13 12:27:04
1     2012-01-13 15:46:19
2     2012-01-13 17:21:55
3     2012-01-13 20:29:59
4     2012-01-12 12:49:53
              ...        
429   2012-12-19 06:31:22
430   2012-12-27 04:00:00
431   2012-12-27 05:30:15
432   2012-12-27 07:00:46
433   2012-12-06 06:34:26
Name: departure, Length: 9918, dtype: datetime64[ns]

In [11]:
datetime.strptime("2012-01-13 00:00", "%Y-%m-%d %H:%M") - timedelta(minutes=111517920)

datetime.datetime(1800, 1, 1, 0, 0)

In [60]:
datetime(1800, 1, 1, 0, 0) + timedelta(minutes=111988110)

datetime.datetime(2012, 12, 4, 12, 30)

In [84]:
td = datetime(2000, 1, 1, 0, 0) - datetime(1800, 1, 1, 0, 0)

In [89]:
td.total_seconds()/60

105189120.0

In [8]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [16]:
start_date = date(2012, 7, 4)
end_date = date(2012, 7, 5)
for single_date in daterange(start_date, end_date):
	d = single_date.strftime("%Y-%m-%d") # 18 - vlaga, 26 - padavine, 21 - hitrost vetra, 15 - temperatura :: 26,21,15,18
	data = requests.get(f"https://meteo.arso.gov.si/webmet/archive/data.xml?lang=si&vars=26,21,15,18&group=halfhourlyData0&type=halfhourly&id=1828&d1={d}&d2={d}&nocache=l1ttpzgh6n69qgtaxx4")
	# save to file
	if data.status_code == 200:
		with open(f"data/{d}.txt", "wb") as f:
			f.write(data.content)
		print(f"{d} - saved")
	else:
		print(f"{d} - {data.status_code}")

2012-07-04 - saved


In [17]:
start_date = date(2012, 1, 1)
end_date = date(2013, 1, 1)
for single_date in daterange(start_date, end_date):
	d = single_date.strftime("%Y-%m-%d")
	# does file exist?
	if not os.path.isfile(f"data/{d}.txt"):
		print(f"{d} - missing")

In [29]:
data = requests.get("https://meteo.arso.gov.si/webmet/archive/data.xml?lang=si&vars=26,15&group=halfhourlyData0&type=halfhourly&id=1828&d1=2012-01-01&d2=2012-01-01&nocache=l1ttpzgh6n69qgtaxx4")
data.text

'<?xml version="1.0" encoding="UTF-8"?><pujs><![CDATA[AcademaPUJS.set( { baseurl:"./icons/", gen:"ACADEMA.AEP.getGenerators(\'archive\').genTerminData(\'yyyy-MM-dd HH:mm\')", datatype:"halfhourly", o:[ "p0" , "p1" ], params:{p0:{ pid:"15", name:"t2m", s:"povp. T", l:"povpreÄ\x8dna temperatura zraka na 2m (Â°C)", unit:"Â°C"},p1:{ pid:"26", name:"padavine", s:"koliÄ\x8dina padavin", l:"koliÄ\x8dina padavin (mm)", unit:"mm"}}, points:{_1828:{_111500640:{p0:"-0.4", p1:"0"}, _111500670:{p0:"-0.5", p1:"0"}, _111500700:{p0:"-0.5", p1:"0"}, _111500730:{p0:"-0.5", p1:"0"}, _111500760:{p0:"-0.6", p1:"0"}, _111500790:{p0:"-0.6", p1:"0"}, _111500820:{p0:"-0.5", p1:"0"}, _111500850:{p0:"-0.5", p1:"0"}, _111500880:{p0:"-0.5", p1:"0"}, _111500910:{p0:"-0.5", p1:"0"}, _111500940:{p0:"-0.6", p1:"0"}, _111500970:{p0:"-0.5", p1:"0"}, _111501000:{p0:"-0.5", p1:"0"}, _111501030:{p0:"-0.7", p1:"0"}, _111501060:{p0:"-0.6", p1:"0"}, _111501090:{p0:"-0.8", p1:"0"}, _111501120:{p0:"-0.9", p1:"0"}, _111501150:{p

In [61]:
data_dir = Path("data_clean")

weather_data = pd.DataFrame()

for file in data_dir.iterdir():
	if file.is_file():
		d = pd.read_csv(file, sep='\t', index_col=0)
		weather_data = weather_data.append(d)

In [68]:
weather_data.to_csv('weather_data.csv', sep='\t')

In [75]:
weather_data

,t2m,rh,padavine,veter_hitrost
min1800,,,,
111500640,-0.4,92,0.0,1.1
111500670,-0.5,92,0.0,0.9
111500700,-0.5,92,0.0,0.7
111500730,-0.5,92,0.0,0.8
111500760,-0.6,92,0.0,0.7
...,...,...,...,...
112027530,-1.2,96,0.0,0.7
112027560,-1.4,96,0.0,1.3
112027590,-1.4,96,0.0,0.6
